# CONDENSE TREE 

In [1]:
import numpy as np
import sklearn.datasets as data

In [2]:
def bfs_from_hierarchy(hierarchy, root):

    to_process = []
    dim = hierarchy.shape[0]
    max_node = 2 * dim
    num_points = max_node - dim + 1
    
    to_process = [int(root)]
    result = []


    while to_process:
        result.extend(to_process)
        to_process = [x - num_points for x in
                        to_process if x >= num_points]
        if to_process:
            
            to_process = hierarchy[to_process,
                                    :2].flatten().astype(np.intp).tolist()
    return result

In [3]:
def condense_tree_complete(hierarchy,min_cluster_size=5):
    
    root = 2 * hierarchy.shape[0]
    num_points = root // 2 + 1
    next_label = num_points + 1


    node_list = bfs_from_hierarchy(hierarchy, root)
    result_list = []
#     print('node_list',node_list)
    ignore = np.zeros(len(node_list), dtype=np.int)
    relabel = np.zeros(root + 1, dtype=np.int)
#     print('relabel_before',relabel)
    relabel[root] = num_points
#     print('relabel_after',relabel)
 
    for node in node_list:
#         result_list = []
#         print(node)
        if node - num_points<0:
            pass
        else:
            children = hierarchy[node - num_points]
            left = int(children[0])
            right = int(children[1])

            if ignore[node] or node < num_points:
#                 print('inside - ignore[node]',ignore[node])
                continue

            INFTY = np.inf

            if children[2] > 0:
                lambda_value = 1.0 / children[2]
            else:
                lambda_value = INFTY

            if left >= num_points:
                left_count = int(hierarchy[left - num_points][3])
            else:
                left_count = 1

            if right >= num_points:
                right_count = int(hierarchy[right - num_points][3])
            else:
                right_count = 1

            if left_count >= min_cluster_size and right_count >= min_cluster_size:
                relabel[left] = next_label
                next_label += 1

                result_list.append((relabel[node], relabel[left], lambda_value,
                                    left_count))
    #             print('1 - ',result_list)
    #             print('length - ',len(result_list))

                relabel[right] = next_label
                next_label += 1

                result_list.append((relabel[node], relabel[right], lambda_value,
                                    right_count))
    #             print('2 - ',result_list)
    #             print('length - ',len(result_list))

            elif left_count < min_cluster_size and right_count < min_cluster_size:
                for sub_node in bfs_from_hierarchy(hierarchy, left):
                    if sub_node < num_points:

                        result_list.append((relabel[node], sub_node,
                                            lambda_value, 1))
    #                     print('3 - ',result_list)
    #                     print('length - ',len(result_list))
                    ignore[sub_node] = True

                for sub_node in bfs_from_hierarchy(hierarchy, right):
                    if sub_node < num_points:

                        result_list.append((relabel[node], sub_node,
                                            lambda_value, 1))
    #                     print('4 - ',result_list)
    #                     print('length - ',len(result_list))
                    ignore[sub_node] = True

            elif left_count < min_cluster_size:
                relabel[right] = relabel[node]
                for sub_node in bfs_from_hierarchy(hierarchy, left):
                    if sub_node < num_points:

                        result_list.append((relabel[node], sub_node,
                                            lambda_value, 1))
    #                     print('5 - ',result_list)
    #                     print('length - ',len(result_list))
                    ignore[sub_node] = True

            else:
                relabel[left] = relabel[node]
                for sub_node in bfs_from_hierarchy(hierarchy, right):
                    if sub_node < num_points:

                        result_list.append((relabel[node], sub_node,
                                            lambda_value, 1))
    #                     print('6 - ',result_list)
    #                     print('length - ',len(result_list))
                    ignore[sub_node] = True

#             for i in range(len(result_list)):
#                 print(result_list[i]) 
                
    return result_list

In [14]:
def bfs_from_hierarchy(hierarchy, root):
    
    print('bfs')
#     print(hierarchy)
#     print(root)

    to_process = []
    dim = hierarchy.shape[0]
    max_node = 2 * dim
    num_points = max_node - dim + 1
    
    to_process = [int(root)]
    result = []


    while to_process:
        result.extend(to_process)
        to_process = [x - num_points for x in
                        to_process if x >= num_points]
        print(to_process)
        print('====')
        if to_process:
            
            to_process = hierarchy[to_process,:2].flatten().astype(np.int).tolist()
            print(to_process)
            
        print(result)
    return result

In [15]:
hierarchy = np.array([
        [ 3,          7,          0.4667349,   2        ],
        [10,          5,          0.51406193,  3        ],
        [ 1,          6,          0.53936375,  2        ],
        [ 2,         11,          0.61524793,  4        ],
        [13,          4,          0.61524793,  5        ],
        [12,          8,          0.69040949,  3        ],
        [ 0,         15,          0.76419459,  4        ],
        [16,          9,          0.76419459,  5        ],
        [17,         14,          2.02808855,  10       ]
    ])

min_cluster_size=5

condense_tree_complete(hierarchy,min_cluster_size=5)

bfs
[8]
====
[17, 14]
[18]
[7, 4]
====
[16, 9, 13, 4]
[18, 17, 14]
[6, 3]
====
[0, 15, 2, 11]
[18, 17, 14, 16, 9, 13, 4]
[5, 1]
====
[12, 8, 10, 5]
[18, 17, 14, 16, 9, 13, 4, 0, 15, 2, 11]
[2, 0]
====
[1, 6, 3, 7]
[18, 17, 14, 16, 9, 13, 4, 0, 15, 2, 11, 12, 8, 10, 5]
[]
====
[18, 17, 14, 16, 9, 13, 4, 0, 15, 2, 11, 12, 8, 10, 5, 1, 6, 3, 7]
bfs
[6]
====
[0, 15]
[16]
[5]
====
[12, 8]
[16, 0, 15]
[2]
====
[1, 6]
[16, 0, 15, 12, 8]
[]
====
[16, 0, 15, 12, 8, 1, 6]
bfs
[]
====
[9]
bfs
[3]
====
[2, 11]
[13]
[1]
====
[10, 5]
[13, 2, 11]
[0]
====
[3, 7]
[13, 2, 11, 10, 5]
[]
====
[13, 2, 11, 10, 5, 3, 7]
bfs
[]
====
[4]


[(10, 11, 0.49307511745480737, 5),
 (10, 12, 0.49307511745480737, 5),
 (11, 0, 1.3085672328562283, 1),
 (11, 8, 1.3085672328562283, 1),
 (11, 1, 1.3085672328562283, 1),
 (11, 6, 1.3085672328562283, 1),
 (11, 9, 1.3085672328562283, 1),
 (12, 2, 1.625361015030152, 1),
 (12, 5, 1.625361015030152, 1),
 (12, 3, 1.625361015030152, 1),
 (12, 7, 1.625361015030152, 1),
 (12, 4, 1.625361015030152, 1)]

# EXPERIMENTATION

In [5]:
def bfs_from_hierarchy(hierarchy, root):

    to_process = []
    dim = hierarchy.shape[0]
    max_node = 2 * dim
    num_points = max_node - dim + 1
    
    to_process = [int(root)]
    result = []


    while to_process:
        result.extend(to_process)
        to_process = [x - num_points for x in
                        to_process if x >= num_points]
        if to_process:
            
            to_process = hierarchy[to_process,
                                    :2].flatten().astype(np.intp).tolist()
    return result

In [6]:
def condense_tree_experiment(hierarchy,min_cluster_size=5):
    count =0
    root = 2 * hierarchy.shape[0]
    num_points = root // 2 + 1
    print('num_points: ',num_points)
    next_label = num_points + 1


    node_list = bfs_from_hierarchy(hierarchy, root)
    result_list = []
    print('node_list',node_list)
    ignore = np.zeros(len(node_list), dtype=np.int)
    relabel = np.zeros(root + 1, dtype=np.int)
    print('relabel_before',relabel)
    relabel[root] = num_points
    print('relabel_after',relabel)
 
    for node in node_list:
        print('==========================================')
        print('node',node)
        if node - num_points<0:
            print('free pass')
            pass
        else:
            count+=1
            children = hierarchy[node - num_points]
            left = int(children[0])
            right = int(children[1])
            print('get children: ', children)
            print('start node(left): ', left)
            print('end node(right): ', right)

            print('current ignore: ',ignore)
            if ignore[node] or node < num_points:
                print('inside - ignore[node]: ',ignore[node])
                continue

            INFTY = np.inf

            if children[2] > 0: # to make sure weights are not negative
                lambda_value = 1.0 / children[2]
            else:
                lambda_value = INFTY

            if left >= num_points:
                left_count = int(hierarchy[left - num_points][3])
                print('left >= num_points: ',left_count)
            else:
                left_count = 1

            if right >= num_points:
                right_count = int(hierarchy[right - num_points][3])
                print('right >= num_points: ',right_count)
            else:
                right_count = 1

            if left_count >= min_cluster_size and right_count >= min_cluster_size:
                relabel[left] = next_label
                next_label += 1

                result_list.append((relabel[node], relabel[left], lambda_value,
                                    left_count))
    #             print('1 - ',result_list)
    #             print('length - ',len(result_list))

                relabel[right] = next_label
                next_label += 1

                result_list.append((relabel[node], relabel[right], lambda_value,
                                    right_count))
    #             print('2 - ',result_list)
    #             print('length - ',len(result_list))

            elif left_count < min_cluster_size and right_count < min_cluster_size:
                for sub_node in bfs_from_hierarchy(hierarchy, left):
                    if sub_node < num_points:

                        result_list.append((relabel[node], sub_node,
                                            lambda_value, 1))
    #                     print('3 - ',result_list)
    #                     print('length - ',len(result_list))
                    ignore[sub_node] = True

                for sub_node in bfs_from_hierarchy(hierarchy, right):
                    if sub_node < num_points:

                        result_list.append((relabel[node], sub_node,
                                            lambda_value, 1))
    #                     print('4 - ',result_list)
    #                     print('length - ',len(result_list))
                    ignore[sub_node] = True

            elif left_count < min_cluster_size:
                relabel[right] = relabel[node]
                for sub_node in bfs_from_hierarchy(hierarchy, left):
                    if sub_node < num_points:

                        result_list.append((relabel[node], sub_node,
                                            lambda_value, 1))
    #                     print('5 - ',result_list)
    #                     print('length - ',len(result_list))
                    ignore[sub_node] = True

            else:
                relabel[left] = relabel[node]
                for sub_node in bfs_from_hierarchy(hierarchy, right):
                    if sub_node < num_points:

                        result_list.append((relabel[node], sub_node,
                                            lambda_value, 1))
    #                     print('6 - ',result_list)
    #                     print('length - ',len(result_list))
                    ignore[sub_node] = True

#             for i in range(len(result_list)):
#                 print(result_list[i]) 
    print('count',count)   
    return result_list

In [7]:
hierarchy = np.array([
        [ 3,          7,          0.4667349,   2        ],
        [10,          5,          0.51406193,  3        ],
        [ 1,          6,          0.53936375,  2        ],
        [ 2,         11,          0.61524793,  4        ],
        [13,          4,          0.61524793,  5        ],
        [12,          8,          0.69040949,  3        ],
        [ 0,         15,          0.76419459,  4        ],
        [16,          9,          0.76419459,  5        ],
        [17,         14,          2.02808855,  10       ],
    
    ])

min_cluster_size=3

condense_tree_experiment(hierarchy,min_cluster_size=5)

num_points:  10
node_list [18, 17, 14, 16, 9, 13, 4, 0, 15, 2, 11, 12, 8, 10, 5, 1, 6, 3, 7]
relabel_before [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
relabel_after [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10]
node 18
get children:  [17.         14.          2.02808855 10.        ]
start node(left):  17
end node(right):  14
current ignore:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
left >= num_points:  5
right >= num_points:  5
node 17
get children:  [16.          9.          0.76419459  5.        ]
start node(left):  16
end node(right):  9
current ignore:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
left >= num_points:  4
node 14
get children:  [13.          4.          0.61524793  5.        ]
start node(left):  13
end node(right):  4
current ignore:  [1 1 0 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0]
left >= num_points:  4
node 16
get children:  [ 0.         15.          0.76419459  4.        ]
start node(left):  0
end node(right):  15
current ignore:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 

[(10, 11, 0.49307511745480737, 5),
 (10, 12, 0.49307511745480737, 5),
 (11, 0, 1.3085672328562283, 1),
 (11, 8, 1.3085672328562283, 1),
 (11, 1, 1.3085672328562283, 1),
 (11, 6, 1.3085672328562283, 1),
 (11, 9, 1.3085672328562283, 1),
 (12, 2, 1.625361015030152, 1),
 (12, 5, 1.625361015030152, 1),
 (12, 3, 1.625361015030152, 1),
 (12, 7, 1.625361015030152, 1),
 (12, 4, 1.625361015030152, 1)]

In [1]:
# Python3 program for Bidirectional BFS
# Search to check path between two vertices
 
# Class definition for node to
# be added to graph
class AdjacentNode:
     
    def __init__(self, vertex):
         
        self.vertex = vertex
        self.next = None
 
# BidirectionalSearch implementation
class BidirectionalSearch:
     
    def __init__(self, vertices):
         
        # Initialize vertices and
        # graph with vertices
        self.vertices = vertices
        self.graph = [None] * self.vertices
         
        # Initializing queue for forward
        # and backward search
        self.src_queue = list()
        self.dest_queue = list()
         
        # Initializing source and
        # destination visited nodes as False
        self.src_visited = [False] * self.vertices
        self.dest_visited = [False] * self.vertices
         
        # Initializing source and destination
        # parent nodes
        self.src_parent = [None] * self.vertices
        self.dest_parent = [None] * self.vertices
         
    # Function for adding undirected edge
    def add_edge(self, src, dest):
         
        # Add edges to graph
         
        # Add source to destination
        node = AdjacentNode(dest)
        node.next = self.graph[src]
        self.graph[src] = node
 
        # Since graph is undirected add
        # destination to source
        node = AdjacentNode(src)
        node.next = self.graph[dest]
        self.graph[dest] = node
         
    # Function for Breadth First Search
    def bfs(self, direction = 'forward'):
         
        if direction == 'forward':
             
            # BFS in forward direction
            current = self.src_queue.pop(0)
            connected_node = self.graph[current]
             
            while connected_node:
                vertex = connected_node.vertex
                 
                if not self.src_visited[vertex]:
                    self.src_queue.append(vertex)
                    self.src_visited[vertex] = True
                    self.src_parent[vertex] = current
                     
                connected_node = connected_node.next
        else:
             
            # BFS in backward direction
            current = self.dest_queue.pop(0)
            connected_node = self.graph[current]
             
            while connected_node:
                vertex = connected_node.vertex
                 
                if not self.dest_visited[vertex]:
                    self.dest_queue.append(vertex)
                    self.dest_visited[vertex] = True
                    self.dest_parent[vertex] = current
                     
                connected_node = connected_node.next
                 
    # Check for intersecting vertex
    def is_intersecting(self):
         
        # Returns intersecting node
        # if present else -1
        for i in range(self.vertices):
            if (self.src_visited[i] and
                self.dest_visited[i]):
                return i
                 
        return -1
 
    # Print the path from source to target
    def print_path(self, intersecting_node,
                   src, dest):
                        
        # Print final path from
        # source to destination
        path = list()
        path.append(intersecting_node)
        i = intersecting_node
         
        while i != src:
            path.append(self.src_parent[i])
            i = self.src_parent[i]
             
        path = path[::-1]
        i = intersecting_node
         
        while i != dest:
            path.append(self.dest_parent[i])
            i = self.dest_parent[i]
             
        print("*****Path*****")
        path = list(map(str, path))
         
        print(' '.join(path))
     
    # Function for bidirectional searching
    def bidirectional_search(self, src, dest):
         
        # Add source to queue and mark
        # visited as True and add its
        # parent as -1
        self.src_queue.append(src)
        self.src_visited[src] = True
        self.src_parent[src] = -1
         
        # Add destination to queue and
        # mark visited as True and add
        # its parent as -1
        self.dest_queue.append(dest)
        self.dest_visited[dest] = True
        self.dest_parent[dest] = -1
 
        while self.src_queue and self.dest_queue:
             
            # BFS in forward direction from
            # Source Vertex
            self.bfs(direction = 'forward')
             
            # BFS in reverse direction
            # from Destination Vertex
            self.bfs(direction = 'backward')
             
            # Check for intersecting vertex
            intersecting_node = self.is_intersecting()
             
            # If intersecting vertex exists
            # then path from source to
            # destination exists
            if intersecting_node != -1:
                print(f"Path exists between {src} and {dest}")
                print(f"Intersection at : {intersecting_node}")
                self.print_path(intersecting_node,
                                src, dest)
                exit(0)
        return -1
 


In [2]:
# Driver code

     
# Number of Vertices in graph
n = 15

# Source Vertex
src = 0

# Destination Vertex
dest = 14

# Create a graph
graph = BidirectionalSearch(n)
graph.add_edge(0, 4)
graph.add_edge(1, 4)
graph.add_edge(2, 5)
graph.add_edge(3, 5)
graph.add_edge(4, 6)
graph.add_edge(5, 6)
graph.add_edge(6, 7)
graph.add_edge(7, 8)
graph.add_edge(8, 9)
graph.add_edge(8, 10)
graph.add_edge(9, 11)
graph.add_edge(9, 12)
graph.add_edge(10, 13)
graph.add_edge(10, 14)

out = graph.bidirectional_search(src, dest)

# if out == -1:
#     print(f"Path does not exist between {src} and {dest}")



Path exists between 0 and 14
Intersection at : 7
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 7
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 6
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 6
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 6
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 4
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 2
*****Path*****
0 4 6 5 2 5 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 0
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 0
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 0
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 0
*****Path*****
0 4 6 7 8 10 14
Path exists between 0 and 14
Intersection at : 0
*****Path*****
0 4 6 7 8 10 14


In [2]:
import queue

class Node:
  def __init__(self, val):
    self.val = val
    self.neighbors = None
    self.visited_right = False  # whether the node was reached by the BFS that started from source
    self.visited_left = False  # whether the node was reached by the BFS that started from destination
    self.parent_right = None  # used for retrieving the final path from start to the meeting point
    self.parent_left = None  # used for retrieving the final path from the meeting point to destination


def bidirectional_search(s, t):

  def extract_path(node):
    """return the path when both BFS's have met"""
    node_copy = node
    path = []

    while node:
      path.append(node.val)
      node = node.parent_right

    path.reverse()
    del path[-1]  # because the meeting node appears twice

    while node_copy:
      path.append(node_copy.val)
      node_copy = node_copy.parent_left
    return path


  q = queue.Queue()
  q.put(s)
  q.put(t)
  s.visited_right = True
  t.visited_left = True

  while not q.empty():
    n = q.get()

    if n.visited_left and n.visited_right:  # if the node visited by both BFS's
      return extract_path(n)

    for node in n.neighbors:
      if n.visited_left == True and not node.visited_left:
        node.parent_left = n
        node.visited_left = True
        q.put(node)
      if n.visited_right == True and not node.visited_right: 
        node.parent_right = n
        node.visited_right = True
        q.put(node)

  return False

n0 = Node(0)
n1 = Node(1)
n2 = Node(2)
n3 = Node(3)
n4 = Node(4)
n5 = Node(5)
n6 = Node(6)
n7 = Node(7)
n0.neighbors = [n1, n5]
n1.neighbors = [n0, n2, n6]
n2.neighbors = [n1]
n3.neighbors = [n4, n6]
n4.neighbors = [n3]
n5.neighbors = [n0, n6]
n6.neighbors = [n1, n3, n5, n7]
n7.neighbors = [n6]
print(bidirectional_search(n0, n7))

[0, 1, 6, 7]


In [ ]:
print(bidirectional_search(n0, n7))